# Advanced Recipe: Directly Comparing Two Reforms
This is an advanced recipe that should be followed only after mastering the basic recipe. This recipe shows how to compare two reforms (instead of comparing a reform to current-law policy) and also shows how to use the reform files available on the Tax-Calculator website (instead of reform files on your computer's disk).

Specifically, we compare the House's TCJA with the conference committee version.

**Ingredients**

No ingredients required because we read reform files from the Tax-Calculator website.

### Imports.

In [1]:
from __future__ import print_function  # Necessary only if using Python 2.7.
import taxcalc as tc
import pandas as pd
import urllib as url_lib  # On Python 3.6 use "import urllib.request as url_lib".

### Read two reform files from Tax-Calculator website.

In [2]:
BASE_URL = ('https://raw.githubusercontent.com/' +
            'open-source-economics/Tax-Calculator/master/taxcalc/reforms/')
reform1_name = 'TCJA_House_Amended.json'  # TCJA as originally passed by House.
reform2_name = 'TCJA_Reconciliation.json'  # TCJA conference committee bill.
reform1_text = url_lib.urlopen(BASE_URL + reform1_name).read()
reform2_text = url_lib.urlopen(BASE_URL + reform2_name).read()

### Specify `Policy` object for static analysis of `reform1`.

In [3]:
reform1 = tc.Calculator.read_json_param_objects(reform1_text, None)
policy1 = tc.Policy()
policy1.implement_reform(reform1['policy'])
if policy1.reform_errors:
    print(policy1.reform_errors)
    exit(1)

### Specify `Policy` object for static analysis of `reform2`.

In [4]:
reform2 = tc.Calculator.read_json_param_objects(reform2_text, None)
policy2 = tc.Policy()
policy2.implement_reform(reform2['policy'])
if policy2.reform_errors:
    print(policy2.reform_errors)
    exit(1)

### Specify `Calculator` objects using `policy1` and `policy2` and calculate for 2018.

In [5]:
calc1 = tc.Calculator(policy=policy1, records=tc.Records.cps_constructor())
calc1.advance_to_year(2018)
calc1.calc_all()
calc2 = tc.Calculator(policy=policy2, records=tc.Records.cps_constructor())
calc2.advance_to_year(2018)
calc2.calc_all()

You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.
You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.


### Compare aggregate individual income tax revenue in 2018.

In [6]:
iitax_rev1 = calc1.weighted_total('iitax')
iitax_rev2 = calc2.weighted_total('iitax')

### Construct `reform2`-vs-`reform1` difference table with results for income deciles.

Start with printing the raw difference table, then a simplified version.

In [7]:
diff_table = calc1.difference_table(calc2, tax_to_diff='iitax')
diff_table

ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'

Exception ValueError: "Buffer dtype mismatch, expected 'Python object' but got 'long'" in 'pandas._libs.lib.is_bool_array' ignored


,count,tax_cut,perc_cut,tax_inc,perc_inc,mean,tot_change,share_of_change,perc_aftertax,pc_aftertaxinc
0,"16,989,416.58","1,108,822.17",6.53,"78,507.66",0.46,-4.88,"-82,939,964.06",0.10,1.57,-1.57
1,"16,988,285.70","2,976,664.47",17.52,"470,086.81",2.77,-16.65,"-282,777,573.02",0.35,-0.17,0.17
2,"16,990,004.05","3,234,769.16",19.04,"4,027,225.94",23.70,6.86,"116,467,193.92",-0.14,0.04,-0.04
3,"16,989,656.05","5,167,627.91",30.42,"4,993,544.06",29.39,-63.11,"-1,072,134,159.29",1.32,-0.26,0.26
4,"16,989,954.66","6,559,859.63",38.61,"6,336,776.63",37.30,-116.32,"-1,976,299,973.04",2.44,-0.38,0.38
5,"16,989,223.86","7,591,363.86",44.68,"6,979,506.87",41.08,-163.03,"-2,769,690,447.94",3.42,-0.41,0.41
6,"16,989,703.32","7,539,405.60",44.38,"8,635,373.34",50.83,-103.23,"-1,753,840,201.44",2.17,-0.20,0.20
7,"16,989,484.52","10,372,491.57",61.05,"6,525,138.59",38.41,-320.51,"-5,445,381,507.33",6.72,-0.47,0.47
8,"16,989,613.44","9,811,367.83",57.75,"7,169,532.73",42.20,-428.31,"-7,276,815,840.60",8.98,-0.44,0.44
9,"16,989,497.48","15,327,615.10",90.22,"1,661,882.38",9.78,"-3,558.27","-60,453,284,348.01",74.64,-1.67,1.67


In [8]:
assert isinstance(diff_table, pd.DataFrame)
diff_extract = pd.DataFrame()
dif_colnames = ['count', 'tax_cut', 'tax_inc',
                'tot_change', 'mean', 'pc_aftertaxinc']
ext_colnames = ['funits(#m)', 'taxfall(#m)', 'taxrise(#m)',
                'agg_diff($b)', 'mean_diff($)', 'aftertax_income_diff(%)']
# Divide by millions or billions as needed.
scaling_factors = [1e-6, 1e-6, 1e-6, 1e-9, 1e0, 1e0, 1e0]
for dname, ename, sfactor in zip(dif_colnames, ext_colnames, scaling_factors):
    diff_extract[ename] = diff_table[dname] * sfactor

### Print total revenue estimates for 2018.

*Estimates in billons of dollars rounded to nearest tenth of a billion.*

In [9]:
print('2018_REFORM1_iitax_rev($B)= {:.1f}'.format(iitax_rev1 * 1e-9))
print('2018_REFORM2_iitax_rev($B)= {:.1f}'.format(iitax_rev2 * 1e-9))

2018_REFORM1_iitax_rev($B)= 1219.4
2018_REFORM2_iitax_rev($B)= 1138.4


### Extract of 2018 income-tax difference table by expanded-income decile.

*`taxfall` is count of `funits` with cut in income tax in reform 2 vs 1.*

*`taxrise` is count of `funits` with rise in income tax in reform 2 vs 1.*


In [10]:
diff_extract

,funits(#m),taxfall(#m),taxrise(#m),agg_diff($b),mean_diff($),aftertax_income_diff(%)
0,16.99,1.11,0.08,-0.08,-4.88,-1.57
1,16.99,2.98,0.47,-0.28,-16.65,0.17
2,16.99,3.23,4.03,0.12,6.86,-0.04
3,16.99,5.17,4.99,-1.07,-63.11,0.26
4,16.99,6.56,6.34,-1.98,-116.32,0.38
5,16.99,7.59,6.98,-2.77,-163.03,0.41
6,16.99,7.54,8.64,-1.75,-103.23,0.20
7,16.99,10.37,6.53,-5.45,-320.51,0.47
8,16.99,9.81,7.17,-7.28,-428.31,0.44
9,16.99,15.33,1.66,-60.45,"-3,558.27",1.67


*Note: deciles are numbered 0-9 with top decile divided into bottom 5%, next 4%, and top 1%, in the lines numbered 11-13, respectively.*